# New Hire - Contribute to HSA

Ensure new hires who enroll in  HDHP will also open and contribute to HSA within 30 days.

Predict  which new hires are likely to enroll in HDHP but are not likely to open and contribute to the HSA within 30 days.

For this analysis and modeling we are solely using *person_hm_choice_enrolled_integrated* (PHCE) table and we have tweaked the "not likely to open and contribute to the HSA within 30 days from *hire date*" to  "not likely to open and contribute to the HSA within 30 days from *first date when they did new hire enrollment in hdhp medical plan*".

-----
In this notebook, we will perform below steps
- Load raw data which is available in S3
- Define number of days to look forward to build target variable. For eg. it is 30 days for given probelm statement
- Pre-process the above data, this will include below items
    - Bring down data to one row per participant (platform_person-intern_id)
    - Build target variable `label` using 30 days logic (X days logic)
- Handle few data exceptions  
- Store pre-processed data in S3, which will further be tranformed using another script

<font color='red'>
We need to edit below three cells before running the whole script.
    
If not editted, data will be pulled and stored from S3 bucket as per given values.
</font>

### For Loading Raw Data 

In [10]:
# Mention the bucket name where raw data is present
bucket='adl-core-sagemaker-studio'

# This is used to seperate different versions of data for whole lifecycle i.e., data pre-process, transformations, modeling and inferences.
# Mention the version whre data is stored in given S3 bucket
version = 'version-1'

# Mention the correct directory where raw-data is present, if data is in multiple files make sure all files are present in this directory
raw_data_path = f'external/artichauhan/HSA/train-data'

# Mention all file names as string in the below list
raw_data_fnames = ['Hype_ML_demographics_2022_06_06_HSA_4x_AM.csv','Hype_ML_demographics_2022_06_06_HSA_3x_AM.csv']

### For Storing Pre-processed Data 

In [11]:
# Mention the bucket name where pre-processed data will be stored. Usually there will no change in bucket name
bucket='adl-core-sagemaker-studio'

# This is used to seperate different versions of data for whole lifecycle i.e., data pre-process, transformations, modeling and inferences.
#Make sure "version" name is as per version of data pre-processing script. Eg. data-pre-processing-v1.ipynb means "version-1"
version = 'version-1'

# Mention the directory where pre-processed data will be stored. Default file name will be "data.csv", we can change it data export step.
pre_processed_data_path = f'external/artichauhan/HSA/train-data/{version}/preprocessed-data'

### For X days logic for creating target variable 

In [12]:
# n-> number of days to look forward and check of participant is contributing in HSA or not.
# At time this use-case was worked upon, it was 30 days, so 'n' is taken as 30
# This value is use at code line 26
n = 30

In [13]:
!pip install --upgrade pandas --index-url https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple

Looking in indexes: https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple


In [14]:
#importing librarires
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import boto3
import io
import time
import gc

%matplotlib inline
pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')

In [15]:
def load_data_from_s3(bucket,version,raw_data_path,raw_data_fnames):
    s3 = boto3.client('s3')
    dataset_nrows = []
    for i, fname in enumerate(raw_data_fnames):
        if i==0:
            print(f'Reading file: {fname}')
            key = f'{raw_data_path}/{fname}'
            obj = s3.get_object(Bucket=bucket, Key=key)
            data = pd.read_csv(io.BytesIO(obj['Body'].read()))
            dataset_nrows.append(data.shape[0])
            print(f'\tFile read successfully | Shape: {data.shape}')
        else:
            print(f'Reading file: {fname}')
            key = f'{raw_data_path}/{fname}'
            obj = s3.get_object(Bucket=bucket, Key=key)
            data2 = pd.read_csv(io.BytesIO(obj['Body'].read()))
            data = data.append(data2,ignore_index=True)
            dataset_nrows.append(data2.shape[0])
            print(f'\tFile read successfully | Shape: {data2.shape}')

    if sum(dataset_nrows) == data.shape[0]:
        print(f'Data from all files loaded successfully | Final Shape: {data.shape}')
        return data.copy()
    else:
        print('There is discrepency in numbers')
        print(f'\tTotal number of rows combined in all files: {sum(dataset_nrows)}')
        print(f'\tAfter combining all files total number of rows are: {data.shape[0]}')
        return None

In [16]:
data = load_data_from_s3(bucket,version,raw_data_path,raw_data_fnames)

Reading file: Hype_ML_demographics_2022_06_06_HSA_4x_AM.csv
	File read successfully | Shape: (1160663, 35)
Reading file: Hype_ML_demographics_2022_06_06_HSA_3x_AM.csv
	File read successfully | Shape: (780960, 35)
Data from all files loaded successfully | Final Shape: (1941623, 35)


In [17]:
data.shape

(1941623, 35)

#### Updating data type of date columns

In [18]:
data['activity_effective_begin_date'] = data['activity_effective_begin_date'].replace('2299-12-31 00:00:00','2261-12-31 00:00:00')
data['activity_effective_begin_date'] = pd.to_datetime(data['activity_effective_begin_date'], format='%Y-%m-%d %H:%M:%S')

In [19]:
data['activity_effective_begin_date'].head()

0   2021-09-01
1   2021-06-25
2   2021-09-30
3   2021-11-26
4   2021-06-01
Name: activity_effective_begin_date, dtype: datetime64[ns]

Since we know that `ee_hsa_contribution_flag` and `er_hsa_contribution_flag` are not reliable. Therefore, we will be creating correct flag columns using respective columns.

In [20]:
data['enrolled_ee_annual_hsa_contribution']
data['ee_hsa_contribution_flag_new'] = 0
data.loc[
    data['enrolled_ee_annual_hsa_contribution'] > 0,
    'ee_hsa_contribution_flag_new'
] = 1

In [21]:
data['enrolled_er_annual_hsa_contribution']
data['er_hsa_contribution_flag_new'] = 0
data.loc[
    data['enrolled_er_annual_hsa_contribution'] > 0,
    'er_hsa_contribution_flag_new'
] = 1

In [22]:
#### Identifying new hires enrolled in any medical plan
# df['new_hire_enrolled_medical_plan'] = 0
# df.loc[
#     (df['mapped_activity_rollup_level_2']=='New Hire Enrollment') &
#     (df['is_no_coverage_option']==0) &
#     (df['dwh_plan_brand_code'].str.contains('MDCL|MDCL-MDCR|RTEE-MDCL')),
#     'new_hire_enrolled_medical_plan'
# ] = 1

#### Identifying new hires enrolled in HDHP medical plan

In [23]:
data['new_hire_enrolled_hdhp_medical_plan'] = 0
data.loc[
    (
        (data['source_system']=='TBA_4X') &
        (data['mapped_activity_rollup_level_2']=='New Hire Enrollment') &
        (data['is_no_coverage_option']==0) &
        (data['dwh_plan_brand_code'].isin(['MDCL','MDCL-MDCR','RTEE-MDCL'])) &
        (data['is_option_hsa_fixed']=='HSA') &
        (data['medical_enrolled_option_type']=='HDHP')
    ) |
    (
        (data['source_system']=='CBA') &
        (data['mapped_activity_rollup_level_2']=='Other Enrollment') &
        (data['is_no_coverage_option']==0) &
        (data['dwh_plan_brand_code'].isin(['MDCL','MDCL-MDCR','RTEE-MDCL'])) &
        (data['is_option_hsa_fixed']=='HSA') &
        (data['medical_enrolled_option_type']=='HDHP')
    ) |
    (
        (data['source_system']=='TBA_3X') &
        (data['mapped_activity_rollup_level_2']=='New Hire Enrollment') &
        (data['is_no_coverage_option']==0) &
        (data['dwh_plan_brand_code']=='HSA')
    ),
    'new_hire_enrolled_hdhp_medical_plan'
] = 1

In [24]:
#unique count of person who enrolled in HDHP medical plan
data[data['new_hire_enrolled_hdhp_medical_plan']==1]['platform_person_internal_id'].nunique()

436719

In [25]:
data['platform_person_internal_id'].nunique()

436719

In [26]:
# #### Identifying new hires enrolled in HDHP medical plan and contributing to HSA - at time of New Hire Enrollment
# data['new_hire_enrolled_hdhp_medical_plan_contributing_hsa'] = 0
# data.loc[
#     (
#         (data['source_system']=='TBA_4X') &
#         (data['mapped_activity_rollup_level_2']=='New Hire Enrollment') &
#         (data['is_no_coverage_option']==0) &
#         (data['dwh_plan_brand_code'].isin(['MDCL','MDCL-MDCR','RTEE-MDCL'])) &
#         (data['is_option_hsa_fixed']=='HSA') &
#         (data['medical_enrolled_option_type']=='HDHP') &
#         (data['ee_hsa_contribution_flag_new']==1)
#     ) |
#     (
#         (data['source_system']=='CBA') &
#         (data['mapped_activity_rollup_level_2']=='Other Enrollment') &
#         (data['is_no_coverage_option']==0) &
#         (data['dwh_plan_brand_code'].isin(['MDCL','MDCL-MDCR','RTEE-MDCL'])) &
#         (data['is_option_hsa_fixed']=='HSA') &
#         (data['medical_enrolled_option_type']=='HDHP') &
#         (data['ee_hsa_contribution_flag_new']==1)
#     ) |
#     (
#         (data['source_system']=='TBA_3X') &
#         (data['mapped_activity_rollup_level_2']=='New Hire Enrollment') &
#         (data['is_no_coverage_option']==0) &
#         (data['dwh_plan_brand_code']=='HSA') &
#         (data['ee_hsa_contribution_flag_new']==1)
#     ),
#     'new_hire_enrolled_hdhp_medical_plan_contributing_hsa'
# ] = 1

### Filtering data for partcipants who had new hire enrollment in HDHP medical plan
Exlcuding person's data who did not had new hire enrollment in HDHP medical plan.

In [27]:
#filtering data for particpants who had new_hire_enrolled_hdhp_medical_plan=1 at some point of time in the given year
ppid_enrolled_hdhp = data[data['new_hire_enrolled_hdhp_medical_plan']==1]['platform_person_internal_id'].unique()
df = data[data['platform_person_internal_id'].isin(ppid_enrolled_hdhp)]
del ppid_enrolled_hdhp
df.shape

(1941623, 38)

In [28]:
#sorting data by participant and activity begin date -> to mark row number for each particant
df = df.sort_values(['platform_person_internal_id','activity_effective_begin_date'],ascending=True)
df.shape

(1941623, 38)

In [29]:
#creating this column to create another column which will duplicate minimum activity begin date corrsoponding to new hire enrollment in
#in HSHP type of medical plan for each participant
df['activity_begin_date_nhehmp'] = pd.NaT
df.loc[df['new_hire_enrolled_hdhp_medical_plan']==1,'activity_begin_date_nhehmp'] = df['activity_effective_begin_date']

In [30]:
#creating column to capture first activity start date corrsponding to new hire enrollment in HDHP medical plan for particpants 
df['min_activity_begin_date_nhehmp'] = df.groupby('platform_person_internal_id')['activity_begin_date_nhehmp'].transform(lambda x: x.min())

In [31]:
#checking for nulls
df['min_activity_begin_date_nhehmp'].isnull().sum()

0

In [32]:
#creating this column to create row number column for each particpant
#this will flag row as 1 if activity_effective_begin_date > min_activity_begin_date_nhehmp
# in case activity_effective_begin_date = min_activity_begin_date_nhehmp then if activity_begin_date_nhehmp not NULL then 1 else 0
#above logic is implemented, in case we have muplitple rows with same date as min_activity_begin_date_nhehmp then to assign flag as 1
#only to rows where new hire enrollment to HDHP medical plan was done
df['activity_begin_date_nhehmp_ge_min'] = 0
df.loc[(df['activity_effective_begin_date']>df['min_activity_begin_date_nhehmp']), 'activity_begin_date_nhehmp_ge_min'] = 1
df.loc[(df['activity_effective_begin_date']==df['min_activity_begin_date_nhehmp']) & (df['activity_begin_date_nhehmp'].notnull()), 'activity_begin_date_nhehmp_ge_min'] = 1

In [ ]:
#assigning row number for each particpant's rows where activity_begin_date_nhehmp_ge_min=1, and we can use row number =1 to capture
#first row for each particpant when theyhad new hire enrollemnt in HDHP medical plan
df['ppid_rnum'] = df[df['activity_begin_date_nhehmp_ge_min']==1].groupby('platform_person_internal_id')['activity_effective_begin_date'].transform(lambda x: range(1,len(x)+1))

In [ ]:
#checking for nulls, having nulls is not a problem. We only need to condier row number = 1 for each particpant
#while brining the dataset to one row per participant 
df['ppid_rnum'].isnull().sum()

#### Creating Label column

In [ ]:
#creating this column to help create label column
#this will check days difference between first new hire enrollment in HDHP medical plan activity date and respective row activity date
#if the difference is less than or equal to specified number 'n' then it flag respective row as 1 else 0
# 'n' is mentioned at start of the script
df['activity_begin_date_within_30d_flag'] = 0
df.loc[(df['activity_effective_begin_date']-df['min_activity_begin_date_nhehmp']).dt.days.isin(range(n+1)), 'activity_begin_date_within_30d_flag']=1
df.shape

In [ ]:
#checking for nulls, we do not expect nulls here
df['activity_begin_date_within_30d_flag'].isnull().sum()

In [ ]:
#finally we are creating label column
#logic: if the particpant contributed in HSA within 30 days of forst new hire enrollment in HDHP mencial plan for given enrolled_plan_year
#not necessaryly at time of enrollment then rows for the particpant will be flaged as 1 else 0
#also there is not checks on other columns while considering ee_hsa_contribution_flag=1
#column will have null values for rows where activity_begin_date_within_30d_flag=0, irrespecitve of particpant contributing or not in HSA
#we can change the look forward days 30 days to any days by changing value of variable 'n' and re-running code for creating
#'activity_begin_date_within_30d_flag' and 'label' columns
df['label'] = df[df['activity_begin_date_within_30d_flag']==1].groupby('platform_person_internal_id')['ee_hsa_contribution_flag_new'].transform(lambda x: x.max())
df.shape

In [ ]:
#droping supporting columns used to create label column
df.drop(
    columns=['activity_begin_date_nhehmp',
             'min_activity_begin_date_nhehmp',
             'activity_begin_date_nhehmp_ge_min',
             'activity_begin_date_within_30d_flag'
            ],
    inplace=True
)

#### Creating new dataframe - one row per person

In [ ]:
#condering only row number 1 for each particpant to bring down data at particpant leve;
df = df[df['ppid_rnum']==1]
df.shape

In [ ]:
#checking unique count of participants
df['platform_person_internal_id'].nunique()

In [ ]:
#chekcing count of particpants contributing in HSA within 30 days of new hire enrollement in HDHP medical plan vs not
# 1 -> contributing, 0 -> not contributing within 30 days 
df['label'].value_counts()

In [ ]:
#reversing 1 <-> 0, not label (target column is align with problem statement)
# 1 -> particpants NOT CONTRINUTING TO HSA within 30 days of first new hire enrollement date in HDHP medical plan
# 0 -> particpants CONTRINUTING TO HSA within 30 days of first new hire enrollement date in HDHP medical plan
df['label'] = df['label'].map({0:1,1:0})
df['label'].value_counts()

In [ ]:
#droping some more columns that were used to create desired dataset
drop_columns = ['ppid_rnum','new_hire_enrolled_hdhp_medical_plan',
                'er_hsa_contribution_flag','ee_hsa_contribution_flag_new','enrolled_plan_year','enrolled_person_plan_efenddt',
                'enrolled_person_plan_efbegdt','activity_effective_begin_date','ee_hsa_contribution_flag',
                'enrolled_ee_annual_hsa_contribution','medical_enrolled_option_type','is_option_hsa_fixed','dwh_plan_brand_code',
                'is_no_coverage_option','mapped_activity_rollup_level_2']

df.drop(
    columns=drop_columns,
    inplace=True
)

In [ ]:
df.rename(columns={'er_hsa_contribution_flag_new':'er_hsa_contribution_flag_new'}, inplace=True)
df.shape

In [ ]:
df.head(3)

### Exporting pre-processed data to S3

In [ ]:
print(f'Uploading pre-processed data here -> s3://{bucket}/{pre_processed_data_path}/data.csv')

df.to_csv(f's3://{bucket}/{pre_processed_data_path}/data.csv', index=False)